# 深層学習ノートブック-5 MLP(多層パーセプトロン)
MNISTデータセットについて、多層パーセプトロンをスクラッチで実装して予測値を算出する

実装方針：
* 隠れ層は1層のみ
* 隠れ層のニューロンの数: 30
    * MNISTでは最終的に１データ当たり10の予測値（確率）が欲しいので、  
    隠れ層のニューロンの数が30の場合、パラメタ$\bm{W}, \bm{b}$のshapeは下記である必要がある。
        * 入力層→隠れ層: 
            * $\bm{W}$: 30 x 特徴量数
            * $\bm{b}$: 1 x 30
        * 隠れ層→出力層: 
            * $\bm{W}$: クラス数(MNISTでは10) x 30
            * $\bm{b}$: 1 x 10
* 隠れ層の活性化関数にはReLUを使用
* モデルの関数を作成し，順伝播（単に入力層→出力層まで左から右に計算すること）で予測した結果を返す  
  今回は逆伝搬（損失を計算してパラメタを更新）はやらない。  

# ●前処理・初期化

In [39]:
import torch
import torch.nn.functional as F  #pytorchの便利関数はFでimportすることが多い。
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
# python debugerをインポート
import pdb

In [40]:
# 変数定義
learning_rate = 0.03
loss_log = []  #学習時の損失記録用のリスト

In [41]:
# データロード
dataset = datasets.load_digits()
feature_names = dataset['feature_names']
X = torch.tensor(dataset['data'], dtype=torch.float32)
target = torch.tensor(dataset['target'])

# shape確認
print(f'shape of X: {X.shape}')
print(X[1])
print('==========================')
print(f'shape of y_train: {target.shape}')
print(target)

shape of X: torch.Size([1797, 64])
tensor([ 0.,  0.,  0., 12., 13.,  5.,  0.,  0.,  0.,  0.,  0., 11., 16.,  9.,
         0.,  0.,  0.,  0.,  3., 15., 16.,  6.,  0.,  0.,  0.,  7., 15., 16.,
        16.,  2.,  0.,  0.,  0.,  0.,  1., 16., 16.,  3.,  0.,  0.,  0.,  0.,
         1., 16., 16.,  6.,  0.,  0.,  0.,  0.,  1., 16., 16.,  6.,  0.,  0.,
         0.,  0.,  0., 11., 16., 10.,  0.,  0.])
shape of y_train: torch.Size([1797])
tensor([0, 1, 2,  ..., 8, 9, 8])


In [42]:
# 目的変数のエンコーディング
y_true = F.one_hot(target, num_classes=10)
print(f'shape of y_true: {y_true.shape}')

shape of y_true: torch.Size([1797, 10])


In [43]:
# 学習データと検証データを8:2に分ける
X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)

print(f'shape of train data: X_train:{X_train.shape}, y_train:{y_train.shape}')
print(f'shape of validation data: X_val:{X_val.shape}, y_val:{y_val.shape}')

shape of train data: X_train:torch.Size([1437, 64]), y_train:torch.Size([1437, 10])
shape of validation data: X_val:torch.Size([360, 64]), y_val:torch.Size([360, 10])


In [44]:
# 学習データ・検証データの標準化。検証データの標準化には学習データの平均、標準偏差を使用することに注意。  
# 今回のようなデータの場合、全体の平均・標準偏差で標準化するでOK
X_mean = X_train.mean()
X_std = X_train.std()
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std

In [67]:
# データ数、特徴量数、隠れ層のノード数、最終的な出力数（ここではクラス数）を定義
m, nf = X_train.shape
nh = 30
n_out = 10

# 入力層→隠れ層の重みW、バイアス項bの初期化
W_1 = torch.randn(size=(nh, nf) ,requires_grad=True) #出力×入力
B_1 = torch.zeros(size=(1, nh), requires_grad=True) # 1 x 出力

# 隠れ層→出力層の重みW、バイアス項bの初期化
W_2 = torch.randn(size=(n_out, nh) ,requires_grad=True) #出力×入力
B_2 = torch.zeros(size=(1, n_out), requires_grad=True) # 1 x 出力

今回は重みの初期化に際し、randではなくrandn(正規分布からの乱数)を用いている。  
またbiasは０で初期化することがおおいので、zerosを使用している。

In [46]:
# softmax関数の実装
def softmax_func(X):
    '''
    X: input tensor.行は各データ、列は各クラスを想定。
    '''
    # x_kが大きすぎると、e^xがinfになるのでmax(x_1, x_2,...,x_K)を各x_kから引く。
    # 各データ（各行）について最大値を求める必要があるので、dimにRank1(列方向で比較)を指定する。
    max_val = X.max(dim=1, keepdim=True).values
    # 各要素のe^xを計算（これが分子になる）
    e_x = (X - max_val).exp()

    # softmax関数の分母の計算
    # 各データについて合計したいので、dim=1を設定。また、分母が0になることを防ぐために分母の式に1e-10を足しておく
    denominator = e_x.sum(dim=1, keepdim=True) + 1e-10

    return e_x / denominator


In [47]:
# 多クラス分類に対応した交差エントロピー
def cross_entropy(y_true, y_pred):
    '''
    y_true: tensor。One-Hot Encoding済みの正解ラベル。
    y_pred: tensor。予測値。softmax関数の出力(0~100%)。
    '''
    # 損失を計算。最終的な損失はスカラーなので、dimを指定する必要はない。
    return - (y_true * torch.log(y_pred + 1e-10)).sum() / y_true.shape[0]


In [69]:
# tensorの線形結合を返す関数
def linear_comb(X, W, B):
    '''
    X, W, B: torch.tensor
    '''
    return X @ W.T + B

In [52]:
# ReLUの実装
def ReLU(Z):
    '''
    Z: torch.tensor
    '''
    # torch.whereによって要素ごとに条件が真・偽のときで別の値を返せる
    # 下記では0より大きい要素はzの値そのままで、0以下は0.になる。
    return torch.where(Z > 0 , Z, 0.)

### ReLUの実装別解
Z.clamp_min(0.)でもOK。  
clamp_min(val)はvalよりも小さい値をvalで置き換える関数。

## MLPをスクラッチで実装

In [76]:
# 隠れ層1層のMLPを実装
def MLP_model(X, W_list, B_list):
    '''
    X: features
    W: List of weights(each edge)
    B: List of Bias(each Layer)
    '''

    # 入力層→隠れ層
    Z1 = linear_comb(X, W_list[0], B_list[0])

    # 活性化関数適用
    A1 = ReLU(Z1)

    # 隠れ層→出力層
    Z2 = linear_comb(A1, W_list[1], B_list[1])

    # 最終出力。ソフトマックス関数を適用して確率の形にする。
    # (実際はこの変換は損失関数計算時にやることが多い)
    A2 = softmax_func(Z2)

    return A2
    

冗長だが今回は各層の処理を一つずつ書いた。  
実際はループにすべし。

In [77]:
# 予測結果算出
W_list = [W_1, W_2]
B_list = [B_1, B_2]

y_train_pred = MLP_model(X_train, W_list=W_list, B_list=B_list)
y_train_pred

tensor([[3.5532e-15, 8.0889e-28, 0.0000e+00,  ..., 3.1388e-27, 3.7840e-08,
         1.0627e-26],
        [2.7773e-06, 3.6767e-24, 3.6031e-37,  ..., 2.8949e-18, 7.4035e-13,
         9.2826e-21],
        [9.7890e-01, 1.6970e-29, 3.2763e-37,  ..., 3.5054e-22, 1.0644e-11,
         2.6484e-22],
        ...,
        [3.6281e-01, 3.1607e-24, 0.0000e+00,  ..., 2.1724e-28, 6.2918e-08,
         5.0386e-41],
        [2.5202e-01, 2.7486e-33, 2.4813e-31,  ..., 7.0383e-12, 2.0191e-08,
         7.7509e-10],
        [1.6545e-16, 2.8026e-45, 0.0000e+00,  ..., 6.7412e-34, 1.2109e-17,
         2.9692e-27]], grad_fn=<DivBackward0>)